In [2]:
import numpy as np 
from numpy.random import seed, randint

http://ajcr.net/Basic-guide-to-einsum/

In [9]:
seed(1643)
A = randint(0, 10, 4).reshape(2, 2)
A

array([[5, 9],
       [6, 2]])

$\sum_{i} A_{i,i}$

In [22]:
np.einsum("ii", A)

7

$\sum_{i} a_i  b_i$

In [26]:
a, b = np.array([1,2,3]), np.array([1, 1, 1])
np.einsum("i,i", a, b)

6

In [33]:
np.einsum("i,j->i", a,b)

array([3, 6, 9])

## Matrix Multiplication

In [3]:
A = np.array([[1, 1, 1],
              [2, 2, 2],
              [5, 5, 5]])

B = np.array([[0, 1, 0],
              [1, 1, 0],
              [1, 1, 1]])

In [8]:
np.einsum("ij,kj -> ik", A, B)

array([[ 1,  2,  3],
       [ 2,  4,  6],
       [ 5, 10, 15]])